In [14]:
import pandas as pd

def process_data(group):
    # 定义评级的顺序
    rating_order = ['BBB', 'A-', 'A', 'A+', 'AA-', 'AA', 'AA+', 'AAA']

    # 计算是否升评
    def is_upgraded(row, prev_row):
        if prev_row is not None and row['rating'] in rating_order and prev_row['rating'] in rating_order:
            return 1 if rating_order.index(row['rating']) > rating_order.index(prev_row['rating']) else 0
        return 0

    # 计算变化率
    group['value_change'] = group['value'].pct_change()

    # 使用 shift 处理前一行
    group['prev_rating'] = group['rating'].shift(1)
    
    # 判断是否升级
    group['is_upgraded'] = group.apply(lambda row: is_upgraded(row, {'rating': row['prev_rating']}), axis=1)
    # for col in group.columns:
    #     if pd.api.types.is_numeric_dtype(group[col]):
    #         group[col + '_change'] = group[col].pct_change()
    # 删除无变化的行和临时列
    # group = group[group['is_upgraded'] == 1].drop(columns=['prev_rating'])

    return group

# 示例数据
data = {
    'ticker': ['AAPL', 'AAPL', 'AAPL', 'AMD', 'AMD', 'AMD'],
    'value': [10, 20, 30, 40, 50, 60],
    'rating': ['BBB', 'A-', 'BBB', 'BBB', 'A-', 'A+']
}

df = pd.DataFrame(data)

# 应用函数到每个 ticker 组
processed_df = df.groupby('ticker').apply(process_data).reset_index(drop=True)

# 显示结果
print(processed_df)

  ticker  value rating  value_change prev_rating  is_upgraded
0   AAPL     10    BBB           NaN        None            0
1   AAPL     20     A-          1.00         BBB            1
2   AAPL     30    BBB          0.50          A-            0
3    AMD     40    BBB           NaN        None            0
4    AMD     50     A-          0.25         BBB            1
5    AMD     60     A+          0.20          A-            1
